In [1]:
import pandas as pd
import xgboost as xgb
import pickle 
model = pickle.load(open('../models/model_xgb.pck', 'rb'))


In [3]:
from datetime import datetime
df_valid = pd.read_hdf('../data/processed/validation/aggr_valid_25feat.hdf')
threshold = datetime(2015, 9, 1)
df_valid['date'] = pd.to_datetime(df_valid['year'].astype(str) + '-' + df_valid['month'].astype(str) + '-1')
df_valid = df_valid[df_valid['date'] >= threshold]

In [4]:
df.head()

,ID,shop_id,item_id,year,month,item_name,item_category_id,item_category_name,first_big_category,last_big_category,...,count_last_week_mall_first_big_category,count_last_week_mall_last_big_category,count_last_week_city_item_id,count_last_week_city_item_category_id,count_last_week_city_first_big_category,count_last_week_city_last_big_category,fridays,days_in_a_month,expected_sales,item_cnt_day
0,0,5,5037,2014,10,"NHL 15 [PS3, русские субтитры]",19,Игры - PS3,Игры,PS3,...,165.0,25.0,0.0,25.0,165.0,25.0,5,31,0.000000,0.0
1,2,5,5233,2014,10,"Need for Speed Rivals (Essentials) [PS3, русск...",19,Игры - PS3,Игры,PS3,...,165.0,25.0,0.0,25.0,165.0,25.0,5,31,0.000000,0.0
2,15,5,4872,2014,10,"Minecraft. Playstation 3 Edition [PS3, русская...",19,Игры - PS3,Игры,PS3,...,165.0,25.0,1.0,25.0,165.0,25.0,5,31,4.428571,3.0
3,21,5,5002,2014,10,"NBA 2K16 [PS3, русская документация]",19,Игры - PS3,Игры,PS3,...,165.0,25.0,0.0,25.0,165.0,25.0,5,31,0.000000,0.0
4,23,5,5814,2014,10,"Plants vs. Zombies Garden Warfare [PS3, русска...",19,Игры - PS3,Игры,PS3,...,165.0,25.0,1.0,25.0,165.0,25.0,5,31,4.428571,2.0


In [4]:
from sklearn import preprocessing
def X(df):
    df['day_of_year'] = df['date'].dt.dayofyear
    date_cols = [
        'month', 'year',
        'fridays', 'day_of_year', 'days_in_a_month'
    ]
    one_hot_cols = [
        'shop_id', 'mall', 'city',
        'item_id', 'item_category_id', 'first_big_category', 'last_big_category',
    ]
    additional = [
        'expected_sales'
    ]

    df_X = df[
        [
            col for col in df.columns
            if col.startswith('count')
        ]
        + date_cols + one_hot_cols + additional
        ]

    # df_X = pd.get_dummies(df_X, columns=one_hot_cols)
    # df_X = pd.get_dummies(df_X, columns=one_hot_cols + date_cols)
    for f in df_X.columns:
        if df_X[f].dtype == 'object':
            lbl = preprocessing.LabelEncoder()
            lbl.fit(list(df_X[f].values))
            df_X[f] = lbl.transform(list(df_X[f].values))
    return df_X

def XY(df):
    df_X = X(df)
    df_y = df['item_cnt_day']
    return df_X, df_y


In [5]:
df_X_valid, df_y_valid = df_valid.drop(['item_cnt_day', 'date'], axis=1), df_valid['item_cnt_day']

In [6]:
from sklearn.metrics import r2_score, mean_squared_error
from math import sqrt
def get_preds_trained_on_all(model, df_X):
    return model.predict(xgb.DMatrix(df_X)).clip(0, 20)


def previous_value_preds(df_X):
    df_X = df_X.reindex(columns=df_X_train.columns, fill_value=0)
    return df_X['count_aggr_1_month_ago_shop_id_item_id'].clip(0, 20)


def validate(model, df_y_valid, df_X_valid):
    df_y_valid = df_y_valid.copy().clip(0, 20)
    df_y_valid_preds = get_preds_trained_on_all(model, df_X_valid)
    # df_y_valid_preds = previous_value_preds(df_X_valid)
    print('r2 score: ', r2_score(df_y_valid, df_y_valid_preds))
    print('RMSE: ', sqrt(mean_squared_error(df_y_valid, df_y_valid_preds)))


In [7]:
validate(model, df_y_valid, df_X_valid)

r2 score:  0.3516786787299254
RMSE:  0.8880534274597472


In [7]:
validate(model, df_y_valid, df_X_valid)

r2 score:  0.3357538987324741
RMSE:  0.8988939337763231


In [21]:

df_test = pd.read_hdf('../data/processed/test/aggr_test_manual_feat.hdf')
df_test_X = X(df_test)
df_test['item_cnt_month'] = get_preds_trained_on_all(model, df_test_X)
df_test[['ID', 'item_cnt_month']].to_csv('../data/submissions/submission_trained_on_year_xgb.csv', index=False)

linreg_test = pd.read_csv('../data/submissions/if_sells_submission_linreg.csv')
print('linreg RMSE: ', sqrt(mean_squared_error(linreg_test['item_cnt_month'], df_test['item_cnt_month'])))
rf_test = pd.read_csv('../data/submissions/if_sells_submission_rf.csv')
print('rf RMSE: ', sqrt(mean_squared_error(rf_test['item_cnt_month'], df_test['item_cnt_month'])))

linreg RMSE:  1.2656266149507325
rf RMSE:  2.0776610156837596


In [8]:

df_test = pd.read_hdf('../data/processed/test/aggr_test.hdf')
df_test_X = X(df_test)
df_test['item_cnt_month'] = get_preds_trained_on_all(model, df_test_X)
df_test[['ID', 'item_cnt_month']].to_csv('../data/submissions/submission_trained_on_year_xgb.csv', index=False)

linreg_test = pd.read_csv('../data/submissions/if_sells_submission_linreg.csv')
print('linreg RMSE: ', sqrt(mean_squared_error(linreg_test['item_cnt_month'], df_test['item_cnt_month'])))
rf_test = pd.read_csv('../data/submissions/if_sells_submission_rf.csv')
print('rf RMSE: ', sqrt(mean_squared_error(rf_test['item_cnt_month'], df_test['item_cnt_month'])))

linreg RMSE:  1.2704333560156416
rf RMSE:  2.0807564524159243


In [12]:

df_test = pd.read_hdf('../data/processed/test/aggr_test.hdf')
df_test_X = X(df_test)
df_test['item_cnt_month'] = get_preds_trained_on_all(model, df_test_X)
df_test[['ID', 'item_cnt_month']].to_csv('../data/submissions/submission_trained_on_year_xgb.csv', index=False)

linreg_test = pd.read_csv('../data/submissions/if_sells_submission_linreg.csv')
print('linreg RMSE: ', sqrt(mean_squared_error(linreg_test['item_cnt_month'], df_test['item_cnt_month'])))
rf_test = pd.read_csv('../data/submissions/if_sells_submission_rf.csv')
print('rf RMSE: ', sqrt(mean_squared_error(rf_test['item_cnt_month'], df_test['item_cnt_month'])))

/home/piotrek/.local/lib/python3.5/site-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


linreg RMSE:  1.2822909100555244
rf RMSE:  2.0878296879063654


In [8]:

df_test = pd.read_hdf('../data/processed/test/aggr_test.hdf')
df_test_X = X(df_test)
df_test['item_cnt_month'] = get_preds_trained_on_all(model, df_test_X)
df_test[['ID', 'item_cnt_month']].to_csv('../data/submissions/submission_trained_on_year_xgb_10.csv', index=False)

linreg_test = pd.read_csv('../data/submissions/if_sells_submission_linreg.csv')
print('linreg RMSE: ', sqrt(mean_squared_error(linreg_test['item_cnt_month'], df_test['item_cnt_month'])))
rf_test = pd.read_csv('../data/submissions/if_sells_submission_rf.csv')
print('rf RMSE: ', sqrt(mean_squared_error(rf_test['item_cnt_month'], df_test['item_cnt_month'])))

/home/piotrek/.local/lib/python3.5/site-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


linreg RMSE:  1.2891728673291027
rf RMSE:  2.0926027080024925


In [12]:
df_test = pd.read_hdf('../data/processed/test/aggr_test_25feat.hdf')
df_test['item_cnt_month'] = get_preds_trained_on_all(model, df_test.drop('ID', axis=1))
df_test[['ID', 'item_cnt_month']].to_csv('../data/submissions/submission_trained_on_year_xgb_10_25feat.csv', index=False)

linreg_test = pd.read_csv('../data/submissions/if_sells_submission_linreg.csv')
print('linreg RMSE: ', sqrt(mean_squared_error(linreg_test['item_cnt_month'], df_test['item_cnt_month'])))
rf_test = pd.read_csv('../data/submissions/if_sells_submission_rf.csv')
print('rf RMSE: ', sqrt(mean_squared_error(rf_test['item_cnt_month'], df_test['item_cnt_month'])))

linreg RMSE:  1.2832880455515667
rf RMSE:  2.088641118528103
